In [ ]:
# =============================================================================
# Cell 1: Imports and Load Data
# =============================================================================
import sys
import os
sys.path.insert(0, './era5vis-main')

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import importlib

# Import terrain, wind, and dynamics modules (reload to get latest changes)
from era5vis import terrain, wind, dynamics
importlib.reload(terrain)
importlib.reload(wind)
importlib.reload(dynamics)

# Load ERA5 data
era5_path = './era5vis-main/era5vis/data/era5_example_dataset.nc'
era5_full = xr.open_dataset(era5_path)

# Select Alps region
era5_data = era5_full.sel(
    latitude=slice(48, 45),
    longitude=slice(6, 16)
)

# Load terrain data (already cached)
terrain_ds = terrain.load_terrain_aspect_dataset("./era5vis-main/era5vis/terrain_cache/terrain_aspect_1km.nc")

print("✅ Data loaded successfully!")
print(f"\nERA5: {dict(era5_data.sizes)}")
print(f"Terrain: {terrain_ds.sizes['latitude']} × {terrain_ds.sizes['longitude']} @ {terrain_ds.attrs['resolution_m']}m")

ModuleNotFoundError: No module named 'terrain'

In [4]:
# =============================================================================
# Cell 2: Enhance ERA5 with Terrain Data
# =============================================================================

# Add terrain information to ERA5 data
era5_enhanced = terrain.enhance_era5_with_terrain(
    era5_data=era5_data,
    terrain_cache_dir="./terrain_cache",
    terrain_resolution_m=1000
)

print("\n✅ ERA5 enhanced with terrain!")
print("\nNew variables added:")
for var in ['terrain_elevation', 'terrain', 'geopotential_height', 'slope_aspect', 'perpendicular_wind', 'parallel_wind']:
    if var in era5_enhanced:
        print(f"  - {var}: {era5_enhanced[var].dims}")

Loading cached terrain aspect from ./terrain_cache/terrain_aspect_1000m.nc
ERA5 domain: lat(45.0, 48.0), lon(6.0, 16.0)
Terrain elevation range on ERA5 grid: -32089m to 2788m

Terrain intersection summary:
   925 hPa: mean height ~  844m,   164/533 ( 30.8%) below terrain
   850 hPa: mean height ~ 1538m,    56/533 ( 10.5%) below terrain
   700 hPa: mean height ~ 3090m,     0/533 (  0.0%) below terrain
   500 hPa: mean height ~ 5664m,     0/533 (  0.0%) below terrain
   300 hPa: mean height ~ 9281m,     0/533 (  0.0%) below terrain

✅ ERA5 enhanced with terrain!

New variables added:
  - terrain_elevation: ('latitude', 'longitude')
  - terrain: ('valid_time', 'pressure_level', 'latitude', 'longitude')
  - geopotential_height: ('valid_time', 'pressure_level', 'latitude', 'longitude')
  - slope_aspect: ('latitude', 'longitude')
  - perpendicular_wind: ('valid_time', 'pressure_level', 'latitude', 'longitude')
  - parallel_wind: ('valid_time', 'pressure_level', 'latitude', 'longitude')


In [5]:
era5_enhanced

<xarray.Dataset> Size: 995kB
Dimensions:              (valid_time: 10, pressure_level: 5, latitude: 13,
                          longitude: 41)
Coordinates:
  * valid_time           (valid_time) datetime64[ns] 80B 2025-10-01 ... 2025-...
  * pressure_level       (pressure_level) float64 40B 925.0 850.0 ... 300.0
  * latitude             (latitude) float64 104B 48.0 47.75 47.5 ... 45.25 45.0
  * longitude            (longitude) float64 328B 6.0 6.25 6.5 ... 15.75 16.0
    number               int64 8B ...
    expver               (valid_time) <U4 160B ...
Data variables:
    z                    (valid_time, pressure_level, latitude, longitude) float32 107kB ...
    u                    (valid_time, pressure_level, latitude, longitude) float32 107kB ...
    v                    (valid_time, pressure_level, latitude, longitude) float32 107kB ...
    terrain              (valid_time, pressure_level, latitude, longitude) bool 27kB ...
    terrain_elevation    (latitude, longitude) float64 4kB 325.5 324.8 ... 0.0
    geopotential_height  (valid_time, pressure_level, latitude, longitude) float32 107kB ...
    slope_aspect         (latitude, longitude) float64 4kB 0.4185 ... -0.4974
    wind_speed           (valid_time, pressure_level, latitude, longitude) float32 107kB ...
    perpendicular_wind   (valid_time, pressure_level, latitude, longitude) float64 213kB ...
    parallel_wind        (valid_time, pressure_level, latitude, longitude) float64 213kB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-11-05T10:28 GRIB to CDM+CF via cfgrib-0.9.1...

In [6]:
terrain_ds

<xarray.Dataset> Size: 6MB
Dimensions:       (latitude: 232, longitude: 772)
Coordinates:
  * latitude      (latitude) float64 2kB 48.0 47.99 47.97 ... 45.03 45.01 45.0
  * longitude     (longitude) float64 6kB 6.0 6.013 6.026 ... 15.97 15.99 16.0
Data variables:
    elevation     (latitude, longitude) float64 1MB ...
    aspect        (latitude, longitude) float64 1MB ...
    aspect_deg    (latitude, longitude) float64 1MB ...
    slope         (latitude, longitude) float64 1MB ...
    terrain_mask  (latitude, longitude) bool 179kB ...
Attributes:
    resolution_m:       990
    min_elevation:      500.0
    min_slope:          1.0
    description:        Terrain slope aspect dataset computed from SRTM data
    aspect_convention:  Downhill direction in radians, 0=North, π/2=East, ±π=...